# Review : Faster R-CNN

>* edwith 강의 내용과 함께 SH Tsang의 [Review: Faster R-CNN](https://towardsdatascience.com/review-faster-r-cnn-object-detection-f5685cb30202)을 번역한 글입니다.
>
>* [2015 NIPS, Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks](https://papers.nips.cc/paper/5638-faster-r-cnn-towards-real-time-object-detection-with-region-proposal-networks.pdf)

이번 글에서는 __Faster R-CNN__에 대해 알아보자. 기존의 모델(Fast R-CNN, R-CNN)은 region proposals를 만들기 위해 CNN이 아닌 'selective search(SS)' 방법을 사용했다.

Faster R-CNN에서는 하나의 Conv network에서 region proposals과 object detection를 모두 수행할 수 있도록 만들었으며, object detection을 더 빠르게 수행할 수 있게 되었다.

** 딥러닝 object detection에 대해 알기위해서 기존의 시리즈인 R-CNN, Fast R-CNN, Faster R_CNN을 함께 읽어보는 것을 추천한다. 

Faster R-CNN은 2015년 NIPS와 2017 TPAMI에서 논문을 발표했다.

><font color='gray'><i>* Paper main thinking<br>
    >To this end, we introduce novel Region Proposal Networks (RPNs) that share convolutional layers with state-of-the-art object detection networks</font></i>

## Index

1. [Region Proposal Network (RPN)](#a)
2. [Detection Network](#b)
3. [4-Step Alternating Training](#c)
4. [Ablation Study](#d)
5. [Detection Results](#e)
6. [Conclusion](#f)

<a id='a'></a>
## 1. Region Proposals Network(RPN)

R-CNN과 Fast R-CNN은 selectiv search 방법으로 region proposals를 찾고, 이후에 object 분류와 bounding box를 찾는 CNN network를 사용했다(두 방법의 차이점은 R-CNN은 pixel 단계(level)에서 detection CNN을 통과시켰고, Fast R-CNN은 feature map 단계에서 detection CNN을 통과시켰다). 즉 두가지 방법모두 region proposal과 detection network가 분리되어(decoupled) 있었다.

이렇게 네트워크를 분리하는(decoupled) 방법은 좋지않다고한다. 먼저 연산하는 과정에서 병목현상(computational bottleneck)을 발생시키고, SS에서 좋지않은 결과를 detection network에 보낼 경우, detection 자체를 좋지않게 만들기 때문이다. 따라서 서로가 연결되도록(correlated) 구성하는(couple) 방법이 더 좋다.

__Faster R-CNN에서는 RPN에서 CNN을 사용하기 시작하며,__  detection과 공유되도록 이 CNN network를 구성한다. 논문에서는 ZFNet, VGGNet 구조를 사용했으며, 전체적인 과정은 다음과 같다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*e6dx5qzUKWwasIVGSuCyDA.png" width="500">
    <br>
    <font color=gray><i>Faster R-CNN</i><font>
</div>
<br>

전체적인 과정은 각 sliding-window 위치에서, k개의 region proposals를 뽑는 것이다.

1. 먼저, 이미지를 conv layer에 통과시켜 feature map을 얻는다.
2. 다음, RPN을 하기위해 feature map위에서 small network(n×n spatial window)를 sliding 한다.
3. 각 위치에서는 k(k=9)개의 anchor boxes(3 scales of 128, 256 and 512, and 3 aspect ratios of 1:1, 1:2, 2:1)를 뽑고 region proposals에 사용한다.
4. _cls_ layer는 2k개의 scores output을 출력한다. 이 값은 boxes(k개) 안에 object가 있는지 없는지(whether there is object or not) 판단하는 값이다.
5. _reg_ layer는 4k개의 값을 출력한다. 이 값은 boxes(k개)의 좌표를 조정하기(corrdinates) 위한 값(box center coordinates, width and height)으로 사용된다.
6. feature map의 크기가 $W ×H$라면, 총 $WHk$의 anchor를 가진다.

각 윈도우는 저차원의 vector로 mapping하고 두 개의 FC layer로 들어가게 된다(This vector is fed into two sibling fully-connected layers—a box-regression layer (reg) and a box-classification layer (cls))

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*wB3ctS9WGNmw6pP_kjLjgg.png" width="600">
    <br>
    <font color=gray><i>The Output of RPN</i></font>
</div>
<br>

3 scales(128, 256, 512), 3 aspect ratios(1:1, 1:2, 2:1)에서 average proposal size는 다음과 같다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*xbaX4MqolDTnfiMTJpX-Qg.png" width="500">
    <br>
    <font color=gray><i>Average Proposal Sizes</i></font>
</div>
<br>

Faster R-CNN에서 사용하는 loss는 다음과 같다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*ifiqMHGbDsyIE7Zf06dAdA.png" width="400">
    <br>
    <font color=gray><i>RPN Loss Function</i></font>
</div>
<br>

첫번째 term은 object 인지아닌지를 판단하는 2개의 class로 분류하는(binary class label) loss값이다. 논문에서는 두 가지 기준을 충족하는 경우positive value로 부여했다.
1. ground-truth box와 가장 높은 IoU(Intersection over-Union)을 갖는 anchor.
2. ground-truth box와 겹치는 IoU값이 0.7 이상인 anchor.


두번째 term은 object일 때만 사용하는$(i.e. p_i* =1)$ bounding boxes regression loss값이다.

따라서 __RPN network__는 각 위치(location)에 물체를 포함하고 있는지 먼저 확인(pre-check)한다. 그리고나서 해당 위치 및 bounding boxes가 object detection network를 통과하게 하고, bounding box 값을 반환하게(returning) 된다.

대부분의 region이 서로 겹치게 되므로, proposals의 숫자를 줄이기 위해 __non-maximun suppresiion(NMS)__가 사용된다(6000개를 N개(N=300)으로 줄임).

<a id='b'></a>
## 2. Detection Network

RPN을 제외하고 나머지 부분은 Fast R-CNN과 비슷하다. ROI pooling을 먼저 통과시키고, 해당 영역을(pooled area)을 CNN과 두 개의 FC branches(class softmax and bounding box regressor)를 통과시킨다. 

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*67iVyCzqapfB5Nyci_zynw.png" width="600">
    <br>
    <font color=gray><i>Fast R-CNN Detection Network</i></font>
</div>
<br>

<a id='c'></a>
## 3. 4-Step Alternating Training

Conv layers로 feature map을 뽑고, 끝부분에서는 여러개의 output을 출력하기위해 학습과정(training procedure)이 조금 달라진다.

1. imagenet으로 pre-trained된 model로 RPN을 학습시킨다
2. imagenet으로 pre-trained된 model로 separate detection network를 학습시킨다((1)에서 학습시킨 RPN을 사용하며, 아직 conv layer를 공유하지 않는다..softmax와 regressor를 별도로 학습한다는 말인듯하다).
3. 위에서 학습한 detector network로 RPN 학습을 초기화시키고 RPN layer만 학습되도록 한다. 이 때는 conv layers를 공유하도록 수정한다.
4. 연결을 유지한채로 detector network만 학습시킨다.

<a id='d'></a>
## 4. Ablation Study

### 4.1. Region Proposal

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*_WiGCG90bcLVMJl4-q3c4A.png" width="600">
    <br>
    <font color=gray><i></i></font>
</div>
<br>

Conv layer를 공유하지 않고 학습한 경우(alternating training에서 앞의 두단계만 수행)에는 58.7% mAP를 얻었으며, conv layer를 공유하면서 학습했을 때는 59.9% mAP를 달성했다. 따라서 SS나 EB보다 해당 방법이 더 좋다고 할 수 있다.

### 4.2. Scales and Ratios

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*DZofSYET9wmabxKUIFpgdg.png" width="500">
    <br>
    <font color=gray><i></i></font>
</div>
<br>

scale과 ratio를 변경하면서 실험을 진행한 표이다. 3 scales and 3 ratios를 사용했을 때 가장 좋은 결과를 얻을 수 있었다. 3 scales and 1 ratios와 비슷한 수치를 얻었지만, 그래도 3 and 3일 때가 더 좋은 결과를 보여줬다.

### 4.3 λ in Loss Function

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*GbRAOPLLob1XWOoiS4LI2A.png" width="400">
    <br>
    <font color=gray><i></i></font>
</div>
<br>

loss 함수에서 $λ$ 값을 변경하면서 학습한 결과이다. $λ=10$ 일때 가장 좋은 결과를 보여주었다.

<a id='e'></a>
## 5. Detection Results

### 5.1 PASCAL VOC 2007

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1250/1*vhtYJ4sc36Hc5Fxd0IZNJg.png" width="600">
    <br>
    <font color=gray><i>Detailed Results</i></font>
</div>
<br>

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*AbTLCWNxDvXGjr8PjMa6sg.png" width="600">
    <br>
    <font color=gray><i>Overall Results</i></font>
</div>
<br>

COCO, VOC 2007(trainval), VOC 2012 data를 사용해서 학습한 결과, 78.8 mAP의 결과를 보여주었다.

### 5.2 PASCAL VOC 2012

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1250/1*PlkPYAPH0TcU-9FFvRQAFg.png" width="600">
    <br>
    <font color=gray><i>Detailed Results</i></font>
</div>
<br>

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*4fty0BsVFvkYs460wPXYiw.png" width="600">
    <br>
    <font color=gray><i>Overall Results</i></font>
</div>
<br>

COCO, VOC 2007(trainval+test), VOC 2012(trainval) dataset을 사용해서 학습한 결과, 75.9% mAP를 얻었다.

### 5.3 MS COCO

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*N8OmCMdSQJ1fHyq1Z6cBwQ.png" width="600">
    <br>
    <font color=gray><i>Overall Results</i></font>
</div>
<br>

42.1% mAP is obtained with IoU @ 0.5 using COCO train set for training.<br>
21.5% mAP is obtained with IoU from 0.5 to 0.95 with step size of 0.05.

### 5.4 Detection Time

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*v2xKdUizHcO-n2kUkOpGbQ.png" width="600">
    <br>
    <font color=gray><i>Detection Time</i></font>
</div>
<br>

Detection time을 비교했을 때, 기존의 SS을 사용했을 때보다 빠른 속도를 보여준 것을 확인 할 수 있다.

### 5.5. Some Examples

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*IyeVVQKWDf02Jlt2-Cexag.png" width="600">
    <br>
    <font color=gray><i>VOC 2007</i></font>
</div>
<br>

<a id='f'></a>
## 6. Conclusion

논문에서는 효율적이고 더 정확한 region proposal을 생성하는 방법인 Region Proposal Networks(RPNs)를 제안하고 있다. RPN을 사용해서 region proposal의 비용(cost)를 줄였으며, detection network와 하나로 묶어 전체적인 소요시간을 줄일수 있었다.

## References

* [ 2015 NIPS] [Faster R-CNN]<br>
Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks
* [2017 TPAMI] [Faster R-CNN]<br>
Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks
* [2015 ICCV] [Fast R-CNN]<br>
 Fast R-CNN
* [2014 CVPR] [R-CNN]<br>
 Rich feature hierarchies for accurate object detection and semantic segmentation
* [2013 IJCV] [Selective Search]<br>
 Selective Search for Object Recognition